# Involute Gear Tooth Profile Generator

This notebook generates accurate involute gear tooth profiles with configurable parameters.
Features:
- Mathematical generation of involute curves
- Complete gear profile with all teeth
- Matplotlib visualization
- DXF export for CAD software

## 1. Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import math
from dataclasses import dataclass
from typing import Tuple, List
import ezdxf
from ezdxf import colors
from IPython.display import display, HTML

## 2. Gear Parameters Configuration

Easily configure your gear parameters here:

In [ ]:
@dataclass
class GearParameters:
    """Configurable parameters for involute gear generation"""
    
    # Basic Parameters
    number_of_teeth: int = 20          # Number of teeth (z)
    module: float = 5.0                 # Module (m) in mm - determines tooth size
    pressure_angle_deg: float = 20.0    # Pressure angle in degrees (typically 20° or 25°)
    
    # Addendum and Dedendum factors (standard values)
    addendum_factor: float = 1.0        # Addendum = addendum_factor * module
    dedendum_factor: float = 1.25       # Dedendum = dedendum_factor * module
    
    # Profile Shift (0 for standard gears)
    profile_shift: float = 0.0          # Profile shift coefficient (x)
    
    # Tooth thickness factor
    backlash: float = 0.0               # Backlash in mm
    
    # Fillet radius factor
    fillet_radius_factor: float = 0.38  # Fillet radius = factor * module
    
    # Resolution for curves
    points_per_curve: int = 50          # Number of points for involute curve
    
    # Display options
    show_pitch_circle: bool = True
    show_base_circle: bool = True
    show_addendum_circle: bool = True
    show_dedendum_circle: bool = True
    
    def __post_init__(self):
        """Calculate derived parameters"""
        self.pressure_angle = math.radians(self.pressure_angle_deg)
        self.pitch_diameter = self.module * self.number_of_teeth
        self.pitch_radius = self.pitch_diameter / 2
        self.base_radius = self.pitch_radius * math.cos(self.pressure_angle)
        self.addendum = self.addendum_factor * self.module
        self.dedendum = self.dedendum_factor * self.module
        self.addendum_radius = self.pitch_radius + self.addendum
        self.dedendum_radius = self.pitch_radius - self.dedendum
        self.tooth_thickness = (math.pi * self.module / 2) - self.backlash
        self.angular_pitch = 2 * math.pi / self.number_of_teeth

# Create default gear parameters
gear_params = GearParameters()
print(f"Gear Configuration:")
print(f"  Number of teeth: {gear_params.number_of_teeth}")
print(f"  Module: {gear_params.module} mm")
print(f"  Pressure angle: {gear_params.pressure_angle_deg}°")
print(f"  Pitch diameter: {gear_params.pitch_diameter:.2f} mm")
print(f"  Base circle diameter: {gear_params.base_radius * 2:.2f} mm")
print(f"  Addendum circle diameter: {gear_params.addendum_radius * 2:.2f} mm")
print(f"  Dedendum circle diameter: {gear_params.dedendum_radius * 2:.2f} mm")

## 3. Involute Curve Generation Functions

In [ ]:
def involute_point(base_radius: float, angle: float) -> Tuple[float, float]:
    """
    Calculate a point on the involute curve.
    
    Args:
        base_radius: Base circle radius
        angle: Unwrap angle in radians
    
    Returns:
        (x, y) coordinates of the involute point
    """
    x = base_radius * (math.cos(angle) + angle * math.sin(angle))
    y = base_radius * (math.sin(angle) - angle * math.cos(angle))
    return x, y

def generate_involute_curve(gear_params: GearParameters) -> np.ndarray:
    """
    Generate the involute curve from base circle to addendum circle.
    
    Args:
        gear_params: Gear parameters
    
    Returns:
        Array of (x, y) points defining the involute curve
    """
    # Calculate the angle range for the involute
    # Start angle (at base circle)
    theta_start = 0
    
    # End angle (at addendum circle)
    # Using the relationship: r = r_b * sqrt(1 + theta^2)
    theta_end = math.sqrt((gear_params.addendum_radius / gear_params.base_radius) ** 2 - 1)
    
    # Generate points along the involute
    theta_values = np.linspace(theta_start, theta_end, gear_params.points_per_curve)
    points = []
    
    for theta in theta_values:
        x, y = involute_point(gear_params.base_radius, theta)
        points.append([x, y])
    
    return np.array(points)

def rotate_points(points: np.ndarray, angle: float, center: Tuple[float, float] = (0, 0)) -> np.ndarray:
    """
    Rotate points around a center.
    
    Args:
        points: Array of (x, y) points
        angle: Rotation angle in radians
        center: Center of rotation
    
    Returns:
        Rotated points
    """
    cos_angle = math.cos(angle)
    sin_angle = math.sin(angle)
    
    # Translate to origin
    translated = points - np.array(center)
    
    # Rotate
    rotated = np.zeros_like(translated)
    rotated[:, 0] = translated[:, 0] * cos_angle - translated[:, 1] * sin_angle
    rotated[:, 1] = translated[:, 0] * sin_angle + translated[:, 1] * cos_angle
    
    # Translate back
    return rotated + np.array(center)

def mirror_points(points: np.ndarray, axis_angle: float = 0) -> np.ndarray:
    """
    Mirror points across an axis passing through the origin.
    
    Args:
        points: Array of (x, y) points
        axis_angle: Angle of the mirror axis
    
    Returns:
        Mirrored points
    """
    # Rotate to align axis with x-axis
    rotated = rotate_points(points, -axis_angle)
    
    # Mirror across x-axis
    rotated[:, 1] = -rotated[:, 1]
    
    # Rotate back
    return rotate_points(rotated, axis_angle)

## 4. Complete Gear Profile Generation

In [ ]:
def generate_single_tooth(gear_params: GearParameters) -> np.ndarray:
    """
    Generate a single tooth profile.
    
    Args:
        gear_params: Gear parameters
    
    Returns:
        Array of points defining one tooth
    """
    # Generate the involute curve
    involute_curve = generate_involute_curve(gear_params)
    
    # Calculate the angle at the pitch circle
    # Find where involute intersects pitch circle
    pitch_angle = math.sqrt((gear_params.pitch_radius / gear_params.base_radius) ** 2 - 1)
    x_pitch, y_pitch = involute_point(gear_params.base_radius, pitch_angle)
    inv_angle_at_pitch = math.atan2(y_pitch, x_pitch)
    
    # Tooth thickness angle at pitch circle
    tooth_angle = gear_params.tooth_thickness / gear_params.pitch_radius
    
    # Rotation to position the involute correctly
    rotation_angle = tooth_angle / 2 - inv_angle_at_pitch
    
    # Rotate the involute curve
    right_side = rotate_points(involute_curve, rotation_angle)
    
    # Create the left side by mirroring
    left_side = mirror_points(right_side)
    left_side = left_side[::-1]  # Reverse order
    
    # Add dedendum circle arc if needed
    if gear_params.dedendum_radius < gear_params.base_radius:
        # Add root circle arc
        angle_start = math.atan2(left_side[0, 1], left_side[0, 0])
        angle_end = math.atan2(right_side[0, 1], right_side[0, 0])
        
        root_arc_angles = np.linspace(angle_start, angle_end, 10)
        root_arc = np.array([
            [gear_params.dedendum_radius * math.cos(a), 
             gear_params.dedendum_radius * math.sin(a)] 
            for a in root_arc_angles
        ])
        
        # Combine: left side + root arc + right side
        tooth_profile = np.vstack([left_side, root_arc, right_side])
    else:
        # Connect involute curves at base circle
        tooth_profile = np.vstack([left_side, right_side])
    
    return tooth_profile

def generate_full_gear(gear_params: GearParameters) -> List[np.ndarray]:
    """
    Generate the complete gear with all teeth.
    
    Args:
        gear_params: Gear parameters
    
    Returns:
        List of tooth profiles
    """
    # Generate single tooth
    single_tooth = generate_single_tooth(gear_params)
    
    # Generate all teeth by rotation
    teeth = []
    for i in range(gear_params.number_of_teeth):
        angle = i * gear_params.angular_pitch
        rotated_tooth = rotate_points(single_tooth, angle)
        teeth.append(rotated_tooth)
    
    return teeth

# Generate the gear
gear_teeth = generate_full_gear(gear_params)
print(f"Generated {len(gear_teeth)} teeth")

## 5. Visualization with Matplotlib

In [ ]:
def plot_gear(gear_params: GearParameters, teeth: List[np.ndarray], 
              figsize: Tuple[float, float] = (12, 12),
              save_path: str = None):
    """
    Plot the gear using matplotlib.
    
    Args:
        gear_params: Gear parameters
        teeth: List of tooth profiles
        figsize: Figure size
        save_path: Optional path to save the figure
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot teeth
    for tooth in teeth:
        ax.plot(tooth[:, 0], tooth[:, 1], 'b-', linewidth=2)
    
    # Plot reference circles if enabled
    circle_angles = np.linspace(0, 2 * np.pi, 100)
    
    if gear_params.show_pitch_circle:
        pitch_x = gear_params.pitch_radius * np.cos(circle_angles)
        pitch_y = gear_params.pitch_radius * np.sin(circle_angles)
        ax.plot(pitch_x, pitch_y, 'g--', linewidth=1, label='Pitch Circle', alpha=0.5)
    
    if gear_params.show_base_circle:
        base_x = gear_params.base_radius * np.cos(circle_angles)
        base_y = gear_params.base_radius * np.sin(circle_angles)
        ax.plot(base_x, base_y, 'r--', linewidth=1, label='Base Circle', alpha=0.5)
    
    if gear_params.show_addendum_circle:
        add_x = gear_params.addendum_radius * np.cos(circle_angles)
        add_y = gear_params.addendum_radius * np.sin(circle_angles)
        ax.plot(add_x, add_y, 'm--', linewidth=1, label='Addendum Circle', alpha=0.5)
    
    if gear_params.show_dedendum_circle:
        ded_x = gear_params.dedendum_radius * np.cos(circle_angles)
        ded_y = gear_params.dedendum_radius * np.sin(circle_angles)
        ax.plot(ded_x, ded_y, 'c--', linewidth=1, label='Dedendum Circle', alpha=0.5)
    
    # Add center mark
    ax.plot(0, 0, 'k+', markersize=10)
    
    # Set equal aspect ratio and grid
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    
    # Labels and title
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_title(f'Involute Gear Profile\n'
                f'{gear_params.number_of_teeth} Teeth, '
                f'Module = {gear_params.module} mm, '
                f'Pressure Angle = {gear_params.pressure_angle_deg}°')
    
    # Set axis limits
    limit = gear_params.addendum_radius * 1.1
    ax.set_xlim(-limit, limit)
    ax.set_ylim(-limit, limit)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Figure saved to {save_path}")
    
    plt.show()

# Plot the gear
plot_gear(gear_params, gear_teeth)

## 6. DXF Export Functionality

In [ ]:
def export_to_dxf(gear_params: GearParameters, teeth: List[np.ndarray], 
                  filename: str = "gear_profile.dxf",
                  add_circles: bool = True):
    """
    Export the gear profile to DXF format.
    
    Args:
        gear_params: Gear parameters
        teeth: List of tooth profiles
        filename: Output DXF filename
        add_circles: Whether to add reference circles
    """
    # Create a new DXF document
    doc = ezdxf.new('R2010')
    msp = doc.modelspace()
    
    # Add teeth as polylines
    for tooth in teeth:
        # Convert tooth profile to list of tuples
        points = [(float(x), float(y)) for x, y in tooth]
        
        # Create polyline
        polyline = msp.add_lwpolyline(points, close=False)
        polyline.dxf.color = colors.BLUE
        polyline.dxf.lineweight = 50  # 0.5mm lineweight
    
    # Add reference circles if requested
    if add_circles:
        # Pitch circle
        pitch_circle = msp.add_circle(
            center=(0, 0),
            radius=gear_params.pitch_radius
        )
        pitch_circle.dxf.color = colors.GREEN
        pitch_circle.dxf.lineweight = 25
        
        # Base circle
        base_circle = msp.add_circle(
            center=(0, 0),
            radius=gear_params.base_radius
        )
        base_circle.dxf.color = colors.RED
        base_circle.dxf.lineweight = 25
        
        # Addendum circle
        add_circle = msp.add_circle(
            center=(0, 0),
            radius=gear_params.addendum_radius
        )
        add_circle.dxf.color = colors.MAGENTA
        add_circle.dxf.lineweight = 25
        
        # Dedendum circle
        ded_circle = msp.add_circle(
            center=(0, 0),
            radius=gear_params.dedendum_radius
        )
        ded_circle.dxf.color = colors.CYAN
        ded_circle.dxf.lineweight = 25
    
    # Add center point
    msp.add_point((0, 0)).dxf.color = colors.WHITE
    
    # Save the DXF file
    doc.saveas(filename)
    print(f"DXF file saved as '{filename}'")
    print(f"File contains:")
    print(f"  - {gear_params.number_of_teeth} tooth profiles")
    if add_circles:
        print(f"  - Reference circles (pitch, base, addendum, dedendum)")
    print(f"  - Center point marker")
    
    return filename

# Export to DXF
dxf_filename = export_to_dxf(gear_params, gear_teeth, "involute_gear.dxf")

## 7. Interactive Gear Generator

Modify parameters and regenerate the gear:

In [ ]:
def generate_and_display_gear(number_of_teeth=20, module=5.0, pressure_angle_deg=20.0,
                             addendum_factor=1.0, dedendum_factor=1.25,
                             show_circles=True, export_dxf=False):
    """
    Generate and display a gear with specified parameters.
    
    This function provides an easy interface for experimenting with different gear parameters.
    """
    # Create gear parameters
    params = GearParameters(
        number_of_teeth=number_of_teeth,
        module=module,
        pressure_angle_deg=pressure_angle_deg,
        addendum_factor=addendum_factor,
        dedendum_factor=dedendum_factor,
        show_pitch_circle=show_circles,
        show_base_circle=show_circles,
        show_addendum_circle=show_circles,
        show_dedendum_circle=show_circles
    )
    
    # Generate gear teeth
    teeth = generate_full_gear(params)
    
    # Display gear information
    print("="*50)
    print("GEAR SPECIFICATIONS")
    print("="*50)
    print(f"Number of teeth: {params.number_of_teeth}")
    print(f"Module: {params.module} mm")
    print(f"Pressure angle: {params.pressure_angle_deg}°")
    print(f"Pitch diameter: {params.pitch_diameter:.2f} mm")
    print(f"Base diameter: {params.base_radius * 2:.2f} mm")
    print(f"Tip diameter: {params.addendum_radius * 2:.2f} mm")
    print(f"Root diameter: {params.dedendum_radius * 2:.2f} mm")
    print(f"Circular pitch: {math.pi * params.module:.3f} mm")
    print(f"Tooth thickness at pitch: {params.tooth_thickness:.3f} mm")
    print("="*50)
    
    # Plot the gear
    plot_gear(params, teeth)
    
    # Export to DXF if requested
    if export_dxf:
        filename = f"gear_{number_of_teeth}T_m{module}.dxf"
        export_to_dxf(params, teeth, filename)
    
    return params, teeth

# Example: Generate a different gear
params2, teeth2 = generate_and_display_gear(
    number_of_teeth=30,
    module=3.0,
    pressure_angle_deg=25.0,
    export_dxf=True
)

## 8. Gear Mesh Visualization

Visualize two gears in mesh:

In [ ]:
def plot_gear_mesh(params1: GearParameters, params2: GearParameters,
                  rotation1: float = 0, rotation2: float = 0):
    """
    Plot two gears in mesh.
    
    Args:
        params1: Parameters for first gear
        params2: Parameters for second gear
        rotation1: Rotation angle for first gear (radians)
        rotation2: Rotation angle for second gear (radians)
    """
    # Check module compatibility
    if abs(params1.module - params2.module) > 0.001:
        print("Warning: Gears have different modules!")
    
    # Calculate center distance
    center_distance = (params1.pitch_radius + params2.pitch_radius)
    
    # Generate teeth
    teeth1 = generate_full_gear(params1)
    teeth2 = generate_full_gear(params2)
    
    # Apply rotations
    teeth1_rotated = [rotate_points(tooth, rotation1) for tooth in teeth1]
    teeth2_rotated = [rotate_points(tooth, rotation2) for tooth in teeth2]
    
    # Translate second gear
    teeth2_translated = [tooth + np.array([center_distance, 0]) for tooth in teeth2_rotated]
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Plot first gear
    for tooth in teeth1_rotated:
        ax.plot(tooth[:, 0], tooth[:, 1], 'b-', linewidth=2, label='Gear 1' if tooth is teeth1_rotated[0] else '')
    
    # Plot second gear
    for tooth in teeth2_translated:
        ax.plot(tooth[:, 0], tooth[:, 1], 'r-', linewidth=2, label='Gear 2' if tooth is teeth2_translated[0] else '')
    
    # Plot pitch circles
    circle_angles = np.linspace(0, 2 * np.pi, 100)
    
    # Gear 1 pitch circle
    pitch1_x = params1.pitch_radius * np.cos(circle_angles)
    pitch1_y = params1.pitch_radius * np.sin(circle_angles)
    ax.plot(pitch1_x, pitch1_y, 'b--', linewidth=1, alpha=0.3)
    
    # Gear 2 pitch circle
    pitch2_x = params2.pitch_radius * np.cos(circle_angles) + center_distance
    pitch2_y = params2.pitch_radius * np.sin(circle_angles)
    ax.plot(pitch2_x, pitch2_y, 'r--', linewidth=1, alpha=0.3)
    
    # Mark centers
    ax.plot(0, 0, 'b+', markersize=10)
    ax.plot(center_distance, 0, 'r+', markersize=10)
    
    # Settings
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_title(f'Gear Mesh Visualization\n'
                f'Gear 1: {params1.number_of_teeth}T, Gear 2: {params2.number_of_teeth}T\n'
                f'Center Distance: {center_distance:.2f} mm')
    
    plt.show()
    
    # Print mesh information
    gear_ratio = params2.number_of_teeth / params1.number_of_teeth
    print(f"\nMesh Information:")
    print(f"  Center distance: {center_distance:.2f} mm")
    print(f"  Gear ratio: {gear_ratio:.3f} (Gear 2 : Gear 1)")
    print(f"  Module: {params1.module} mm")

# Create two meshing gears
gear1_params = GearParameters(number_of_teeth=15, module=5.0)
gear2_params = GearParameters(number_of_teeth=30, module=5.0)

# Visualize mesh
plot_gear_mesh(gear1_params, gear2_params, rotation1=0, rotation2=math.pi/30)

## 9. Batch Export - Generate Multiple Gear Variations

In [ ]:
def batch_generate_gears(teeth_range, module_values, pressure_angles, export_dxf=True, export_png=True):
    """
    Generate multiple gear variations for comparison or manufacturing.
    
    Args:
        teeth_range: List or range of tooth counts
        module_values: List of module values
        pressure_angles: List of pressure angles
        export_dxf: Export DXF files
        export_png: Export PNG images
    """
    import os
    
    # Create output directory
    os.makedirs('gear_exports', exist_ok=True)
    
    generated_gears = []
    
    for teeth in teeth_range:
        for module in module_values:
            for angle in pressure_angles:
                print(f"\nGenerating: {teeth}T, Module {module}, Angle {angle}°")
                
                # Create gear
                params = GearParameters(
                    number_of_teeth=teeth,
                    module=module,
                    pressure_angle_deg=angle
                )
                
                teeth_profile = generate_full_gear(params)
                
                # Generate filename base
                filename_base = f"gear_{teeth}T_m{module}_a{angle}"
                
                # Export DXF
                if export_dxf:
                    dxf_path = f"gear_exports/{filename_base}.dxf"
                    export_to_dxf(params, teeth_profile, dxf_path, add_circles=False)
                
                # Export PNG
                if export_png:
                    png_path = f"gear_exports/{filename_base}.png"
                    fig, ax = plt.subplots(figsize=(8, 8))
                    for tooth in teeth_profile:
                        ax.plot(tooth[:, 0], tooth[:, 1], 'b-', linewidth=2)
                    ax.set_aspect('equal')
                    ax.grid(True, alpha=0.3)
                    ax.set_title(f'{teeth}T, Module {module}, {angle}°')
                    plt.savefig(png_path, dpi=150, bbox_inches='tight')
                    plt.close()
                    print(f"  Saved: {png_path}")
                
                generated_gears.append({
                    'params': params,
                    'filename_base': filename_base
                })
    
    print(f"\n{'='*50}")
    print(f"Batch generation complete!")
    print(f"Generated {len(generated_gears)} gear variations")
    print(f"Files saved in 'gear_exports' directory")
    
    return generated_gears

# Example batch generation
batch_results = batch_generate_gears(
    teeth_range=[12, 20, 24],
    module_values=[3.0, 5.0],
    pressure_angles=[20.0],
    export_dxf=True,
    export_png=True
)

## 10. Quick Reference - Common Gear Configurations

Here are some quick examples you can run:

In [ ]:
# Small precision gear
small_gear = GearParameters(
    number_of_teeth=12,
    module=1.0,
    pressure_angle_deg=20.0
)

# Standard gear
standard_gear = GearParameters(
    number_of_teeth=20,
    module=5.0,
    pressure_angle_deg=20.0
)

# High pressure angle gear (stronger teeth)
strong_gear = GearParameters(
    number_of_teeth=24,
    module=4.0,
    pressure_angle_deg=25.0
)

# Large gear for high torque
large_gear = GearParameters(
    number_of_teeth=60,
    module=8.0,
    pressure_angle_deg=20.0
)

# Select which gear to visualize
selected_gear = standard_gear  # Change this to try different configurations
teeth = generate_full_gear(selected_gear)
plot_gear(selected_gear, teeth)

# Export the selected gear
export_to_dxf(selected_gear, teeth, "selected_gear.dxf")

## Summary

This notebook provides a complete toolkit for generating involute gear profiles with the following features:

1. **Configurable Parameters**: Easy adjustment of all gear parameters including number of teeth, module, pressure angle, and more.

2. **Accurate Mathematics**: Proper involute curve generation based on gear theory.

3. **Visualization**: Clear matplotlib plots with reference circles and customizable display options.

4. **DXF Export**: Industry-standard CAD format export for manufacturing.

5. **Gear Mesh Visualization**: See how two gears work together.

6. **Batch Generation**: Create multiple gear variations efficiently.

### Tips for Use:

- **Module**: Determines the size of teeth. Larger module = larger teeth.
- **Pressure Angle**: Standard values are 20° or 25°. Higher angles give stronger but noisier gears.
- **Number of Teeth**: Minimum ~12 teeth to avoid undercutting (depends on pressure angle).
- **For Manufacturing**: Export to DXF and import into your CAD software for further refinement.

### Next Steps:

- Add helical gear support
- Implement gear strength calculations
- Add internal gear generation
- Include backlash and tolerance calculations
- Add animation of meshing gears